# Isotonic Convex Expectile Regression (ICER)

- Author: Sheng Dai (sheng.dai@aalto.fi)
- Date  : July 30, 2020

Reference:

[1] Kuosmanen T., Dai S., Zhou X. Partial frontiers are not quantiles. Unpublished working paper (available from the authors by request).

In [1]:
import pandas as pd
import numpy as np

In [2]:
# import the package pystoned
from pystoned import ICQER

In [3]:
# import Finnish electricity distribution firms data
url = 'https://raw.githubusercontent.com/ds2010/pyStoNED-Tutorials/master/Data/firms.csv'
df = pd.read_csv(url, error_bad_lines=False)

In [4]:
# output
y = df['Energy']

# inputs
x1 = df['OPEX']
x1 = np.asmatrix(x1).T
x2 = df['CAPEX']
x2 = np.asmatrix(x2).T
x = np.concatenate((x1, x2), axis=1)

In [6]:
# define and solve the ICER model

tau = 0.5
cet = "addi"
fun = "prod"
rts = "vrs"

model = ICQER.icer(y, x, tau, cet, fun, rts)

# using local solver (MOSEK API)
from pyomo.opt import SolverFactory
opt = SolverFactory("mosek")
results = opt.solve(model, tee=True)

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : QO (quadratic optimization problem)
  Constraints            : 3714            
  Cones                  : 0               
  Scalar variables       : 534             
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Quadratic to conic reformulation started.
Quadratic to conic reformulation terminated. Time: 0.02    
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 35
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :       

In [7]:
# retrive the residuals (eplus)
val = list(model.ep[:].value)
eplus = np.asarray(val)
eplus

array([2.17832849e-02, 7.78432121e-05, 1.62465423e-05, 8.91984861e-07,
       5.24753757e-05, 7.06000722e+01, 1.46607177e-05, 1.55888922e-03,
       8.90500668e-01, 3.17998618e+01, 2.39269090e-02, 2.30100509e-02,
       2.16668411e-02, 3.61762715e-05, 2.03562040e-02, 6.48888333e+01,
       2.17197163e-02, 6.13507605e-05, 7.85417897e-05, 7.48493403e-05,
       2.16323306e-02, 2.87998667e+01, 3.04571557e-04, 3.50020634e+00,
       2.15890796e-02, 1.45064524e-04, 1.38182882e-05, 1.37362991e-06,
       5.71669121e+01, 4.83747786e-05, 1.49227924e-04, 1.95888941e+02,
       4.00000211e+01, 2.16059528e-02, 2.11533794e-02, 1.75888331e-06,
       5.89316664e-06, 2.16688233e-02, 7.25006772e+00, 8.62069186e-05,
       2.16302371e-02, 8.49559231e-05, 1.80000491e+01, 2.15514998e-02,
       2.88991987e+00, 3.89412131e-05, 4.69869189e-06, 5.33846104e-05,
       1.05594898e-05, 5.25031769e+00, 6.24991461e+00, 2.17177105e-04,
       8.89604617e-01, 3.88891934e+00, 2.00042423e+00, 1.03000045e+02,
      

In [8]:
# retrive the residuals (eminus)
val = list(model.em[:].value)
eminus = np.asarray(val)
eminus

array([2.08541411e-02, 3.83350157e+00, 1.91110553e+01, 3.53749974e+02,
       5.75028085e+00, 4.45575389e-06, 2.18331513e+01, 2.01759768e-01,
       3.35124842e-04, 9.84323648e-06, 2.29274598e-02, 2.15160000e-02,
       2.07860717e-02, 8.33348067e+00, 2.11684258e-02, 4.89041097e-06,
       2.15270380e-02, 4.83328908e+00, 3.83355411e+00, 4.00014869e+00,
       2.16268358e-02, 1.08312088e-05, 9.98984334e-01, 8.64994894e-05,
       2.18202610e-02, 2.11174076e+00, 2.21111676e+01, 2.29999966e+02,
       5.27629518e-06, 6.20024163e+00, 2.00033684e+00, 1.60883006e-06,
       7.81637090e-06, 2.14903037e-02, 1.98464265e-02, 1.80111067e+02,
       5.33999781e+01, 2.14864346e-02, 4.06674804e-05, 3.50022185e+00,
       2.17855533e-02, 3.50011010e+00, 1.68175444e-05, 2.16049448e-02,
       1.02611661e-04, 7.75038548e+00, 6.70000159e+01, 5.49992235e+00,
       2.91112329e+01, 5.81811706e-05, 4.70721628e-05, 1.40080394e+00,
       3.34637822e-04, 7.69160381e-05, 1.50123530e-04, 3.06105291e-06,
      

In [9]:
# retrive the beta
ind = list(model.b)
val = list(model.b[:,:].value)
beta= np.asarray([ i + tuple([j]) for i, j in zip(ind, val)]) 

beta = pd.DataFrame(beta, columns = ['Name', 'Key', 'Value'])
beta = beta.pivot(index='Name', columns='Key', values='Value')
beta.columns = ['b1', 'b2']

beta

,b1,b2
Name,,
0.0,5.144780e-03,4.670709e-03
1.0,7.385590e-06,4.373215e-07
2.0,5.626278e-07,7.662624e-08
3.0,1.586293e-09,3.872758e-10
4.0,1.382835e-06,5.386203e-07
...,...,...
84.0,2.462247e-08,2.505183e-08
85.0,5.391407e-03,9.518294e-03
86.0,9.720916e-03,1.043594e-02
